In [1]:
import pickle
import numpy as np
import re
import os

seed = 2023

In [2]:
import torch

# set device
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
import transformers

# set to only report critical errors to avoid excessing logging
transformers.utils.logging.set_verbosity(50)

In [4]:
from sig_networks.scripts.fine_tune_bert_classification import (
    fine_tune_transformer_average_seed,
)

In [5]:
output_dir = "talklife_moc_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

TalkLife MoC

In [6]:
%run load_talklifemoc.py

In [7]:
%run load_sbert-embeddings.py

In [8]:
sbert_embeddings.shape

torch.Size([18604, 384])

Baseline: Fine-tune BERT for classification

In [9]:
num_epochs = 10
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"

In [10]:
# create indices for kfold
fold_col_names = [c for c in df.columns if "fold" in c]
fold_list = []
for foldc in fold_col_names:
    fold_list.append(
        (
            df[df[foldc] == "train"].index,
            df[df[foldc] == "dev"].index,
            df[df[foldc] == "test"].index,
        )
    )
fold_list = tuple(fold_list)

In [11]:
bert_classifier = fine_tune_transformer_average_seed(
    num_epochs=num_epochs,
    pretrained_model_name="bert-base-uncased",
    df=df,
    feature_name="content",
    label_column="label",
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    split_ids=None,
    split_indices=fold_list,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/bert_classifier.csv",
    device=device,
    verbose=False,
)

Map:   0%|          | 0/18604 [00:00<?, ? examples/s]

Map:   0%|          | 0/18604 [00:00<?, ? examples/s]

/import/nlp/tt003/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`label` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
bert_classifier

In [ ]:
bert_classifier["f1"].mean()

In [ ]:
bert_classifier["precision"].mean()

In [ ]:
bert_classifier["recall"].mean()

In [ ]:
np.stack(bert_classifier["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(bert_classifier["precision_scores"]).mean(axis=0)

NameError: name 'bert_classifier' is not defined

In [ ]:
np.stack(bert_classifier["recall_scores"]).mean(axis=0)